### import

In [0]:
# !pip install torch

In [0]:
from matplotlib import  pyplot as plt
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model, datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

# from torchvision import datasets, transforms
import torchvision

# ワインデータ

In [0]:
load_data = datasets.load_wine()
# load_data = datasets.load_iris()

features = pd.DataFrame(data=load_data.data, columns=load_data.feature_names)
targets = pd.DataFrame(data=load_data.target, columns=['class'])

In [0]:
features.head()

In [0]:
targets.head()

In [0]:
len(features)

# スケーリング

In [0]:
if True:
  scaled_features = {}
  for each in features.columns:
      mean, std = features[each].mean(), features[each].std()
      scaled_features[each] = [mean, std]
      features.loc[:, each] = (features[each] - mean)/std
    
features.head()

In [0]:
# Dummy Vriables (PyTorch では不要　CrossEntropyLoss 関数内で処理される)
# targets = pd.get_dummies(targets['class'], prefix=None, drop_first=False)

# 訓練データ

In [0]:
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.1)

train_features_tensor = torch.tensor(train_features.values, dtype=torch.float32)
train_targets_tensor = torch.tensor(train_targets.values.reshape(-1), dtype=torch.long)
train_dataset = torch.utils.data.TensorDataset(train_features_tensor,  train_targets_tensor)

test_features_tensor = torch.tensor(test_features.values, dtype=torch.float32)
test_targets_tensor = torch.tensor(test_targets.values.reshape(-1), dtype=torch.long)
test_dataset = torch.utils.data.TensorDataset(test_features_tensor,  test_targets_tensor)

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=16,
                                            shuffle=True,
                                            num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                            batch_size=16,
                                            shuffle=False, 
                                            num_workers=2)

# 学習モデル 

In [0]:
class Net(torch.nn.Module):
   def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 60)
        self.fc2 = nn.Linear(60, 3)
        self.softmax = nn.Softmax(dim=1)

   def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
#         X = self.softmax(X)
        X = F.log_softmax(X)
        return X

In [0]:
net = Net()
# net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.88, nesterov=True)
# optimizer = optim.Adam(net.parameters(), lr=0.001)

# 学習

In [0]:
losses = {'train':[], 'validation':[]}

for epoch in range(5):  # 精度が悪いときは、カッコの中の数をあげてみましょう。

   running_loss = 0.0
   for i, data in enumerate(train_loader, 0):
       inputs, labels = data
      
#        inputs, labels = inputs.cuda(), labels.cuda()
#        inputs, labels =  Variable(inputs), Variable(labels)

       optimizer.zero_grad()

       outputs = net(inputs)

       loss = criterion(outputs, labels)

       loss.backward()
       optimizer.step()

       # print statistics
       running_loss += loss.item()
       losses['train'].append(loss)
      
#        running_loss += loss.data[0]
       if i % 1000 == 0:
           print('epoch [%d] loss: %.5f' %
                 (epoch + 1, running_loss))
#            print(outputs , end=": ")
#            print(labels)

           running_loss = 0.0

# 学習結果

In [0]:
plt.plot(losses['train'], label='Training loss')
# plt.plot(losses['validation'], label='Validation loss')
plt.legend()
_ = plt.ylim()

In [0]:
for i, data in enumerate(test_loader, 0):
   inputs, labels = data
   inputs, labels =  Variable(inputs), Variable(labels)

# clear the gradient
#    optimizer.zero_grad()
   outputs = net(inputs)
   for j, y in enumerate(outputs , 0):
       pred = y.argmax().tolist()
       label = labels[j].tolist()
       print(pred , end=" -> " )
       print(label , end = ":  ")
       if pred  == label:
         print("Ok")
       else:
         print("NG")
   
